<a href="https://colab.research.google.com/github/Youngprize33/Python_Algorithm/blob/main/%EC%BD%94%EB%A1%9C%EB%82%98_%EB%8D%B0%EC%9D%B4%ED%84%B0_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : 'your_service_key', quote_plus('startCreateDt') : '20200410', quote_plus('endCreateDt') : '20210831' })

#request = Request(url + queryParams)
response = requests.get(url + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
# xmlobj    # 디버깅용.

rows = xmlobj.findAll('item')
# rows    # 디버깅용.
columns = rows[0].find_all()
# columns    # 디버깅용.
# columns[0].name    # 디버깅용.
# columns[0].text    # 디버깅용.

# 모든 행과 열의 값을 모아 매트릭스로 만들어보자.
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
        if i == 0:
            nameList.append(columns[j].name)
        # 컬럼값은 모든 행의 값을 저장해야한다.    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
    
result = pd.DataFrame(rowList, columns=nameList)
result.head()


,createDt,deathCnt,defCnt,gubun,gubunCn,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,seq,stdDay,updateDt
0,2021-08-31 09:43:52.857,13,5836,검역,隔離區,Lazaretto,13,5312,511,0,13,-,11846,2021년 08월 31일 00시,null
1,2021-08-31 09:43:52.857,2,2602,제주,济州,Jeju,9,2330,270,8,1,385.69,11845,2021년 08월 31일 00시,null
2,2021-08-31 09:43:52.857,31,9980,경남,庆南,Gyeongsangnam-do,56,9091,858,56,0,298.78,11844,2021년 08월 31일 00시,null
3,2021-08-31 09:43:52.857,88,7100,경북,庆北,Gyeongsangbuk-do,27,6394,618,27,0,269,11843,2021년 08월 31일 00시,null
4,2021-08-31 09:43:52.857,19,2600,전남,全南,Jeollanam-do,14,2375,206,13,1,140.42,11842,2021년 08월 31일 00시,null


In [ ]:
# \xa0, \xa9 를 없애줌
result = result.applymap(lambda x: x.replace('\xa0','').replace('\xa9',''))  

# encoding: cp949 로 전환--> 엑셀에서 깨지지 않게
result.to_csv('covid-19', encoding='utf-8')